In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("IngestaoBronze") \
    .getOrCreate()

In [0]:
%run ./Intancia_Containers

In [0]:
# Lista para armazenar dicionários com os metadados
lista_arquivos_processada = []

for nome, obj in containers_registry.items():
    # Obtém a lista de arquivos do container
    arquivos = obj.get_files_list()
    
    for arq in arquivos:
        lista_arquivos_processada.append({
            "path": arq.path,
            "name": arq.name,
            "size": arq.size,
            "modificationTime": arq.modificationTime,
            "container_name": nome
        })

    print(f"✅ Metadados coletados de: {nome.upper()}")

# Criando o DataFrame com os arquivos
df_inventario = spark.createDataFrame(lista_arquivos_processada)

# Convertendo modificationTime para um formato mais amigável
df_inventario = df_inventario.withColumn(
    "modificationTime", 
    (F.col("modificationTime") / 1000).cast("timestamp")
)

display(df_inventario)